In [ ]:
df=pd.read_csv('D:/data/train.csv')

缺失值观察

In [ ]:
df.info()

In [ ]:
df.isnull() or pd.isnull(df)       #返回布尔值；若数据缺失则为True，反之False

In [ ]:
df.notnull()      #isnull()反义

In [ ]:
df.isnull().sum() #可以反映各列缺失数据总和

In [ ]:
df['Age'].isnull().sum()     #Age那一列缺失数据个数

缺失值过滤

In [ ]:
#过滤掉缺失数据    （丢弃所有包括缺失值的行或列）
df.dropna()

In [ ]:
#过滤掉某列缺失数据
df['Age'].dropna()

In [ ]:
df[df['Age'].notnull()] #只输出没有缺失数据的部分

In [ ]:
#丢弃全为NA的那些行
df.dropna(how='all')

In [ ]:
#丢弃全为NA的那些列
df.dropna(axis=1, how='all')
#丢弃但凡有NA的那些列
df.dropna(axis=1, how='any')

In [ ]:
#留下还有一部分观测数据的行，用thresh参数；thresh（）括号内为可用参数数量
df.dropna(thresh=2)

缺失值填充

In [ ]:
#用0填充NA
df.fillna(0)   

In [ ]:
#用C1填充Cabin列的所有NA
df.fillna({'Cabin':'C1'})

In [ ]:
#用NA行上面一行的数据填充该行NA值
df.fillna(method='ffill')
#用NA行上面一行的数据填充该行NA值，最多可连续两次
df.fillna(method='ffill', limit=2)

In [ ]:
#用这一行的平均数来填充该行的所有NA值
df['Age'].fillna(df['Age'].mean())
df.fillna(df.mean())

重复值查看

In [ ]:
df.duplicated() #某行与上一行重复，会输出True; 不然是False

In [ ]:
#去掉重复行
df.drop_duplicates()

In [ ]:
#去掉特定重复行
df.drop_duplicates(['Age'])

特征观察与处理

我们对特征进行一下观察，可以把特征大概分为两大类：
数值型特征：Survived ，Pclass， Age ，SibSp， Parch， Fare，其中Survived， Pclass为离散型数值特征，Age，SibSp， Parch， Fare为连续型数值特征
文本型特征：Name， Sex， Cabin，Embarked， Ticket，其中Sex， Cabin， Embarked， Ticket为类别型文本特征。
数值型特征一般可以直接用于模型的训练，但有时候为了模型的稳定性及鲁棒性会对连续变量进行离散化。
文本型特征往往需要转换成数值型特征才能用于建模分析。

分箱：分类

In [ ]:
df1=pd.cut(df,bins,labels=['1','2','3'], right=False) #闭合端在左边 right=false

In [ ]:
#平均分成5份
df=pd.read_csv('D:/data/train.csv')
df['Ageband']=pd.cut(df['Age'],5,labels=['1','2','3','4','5'])    

In [ ]:
#按(0,5],(5,15],(15,30],(30,50],(50,80]分类
df=pd.read_csv('D:/data/train.csv')
bins=[0,5,15,30,50,80]
df['Ageband']=pd.cut(df['Age'],bins,labels=['1','2','3','4','5']) 

In [ ]:
#按10%，30%，50%，70%，90%分类！按分数分类法需用pd.qcut
df=pd.read_csv('D:/data/train.csv')
bins=[0,0.1,0.3,0.5,0.7,0.9]
df['Ageband']=pd.qcut(df['Age'],bins,labels=['1','2','3','4','5'])
print (df.head(3))

In [ ]:
#若需要count某一类有多少数据
pd.value_counts(df['Ageband'])
or
df['Ageband'].value_counts()

In [ ]:
df['Sex'].unique()
Output: array(['male', 'female'], dtype=object)

将文本变量用数值变量表示：
map：
df['Sextype']=df['Sex'].map({'male':1,'female':2}) 
replace：
df['Sextype']=df['Sex'].replace(['male','female'],['1','2'])

In [3]:
#例如：
import pandas as pd
df=pd.read_csv('D:/data/train.csv')
df['Sextype']=df['Sex'].map({'male':1,'female':2}) 
print (df.head(3))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  Sextype  
0      0         A/5 21171   7.2500   NaN        S        1  
1      0          PC 17599  71.2833   C85        C        2  
2      0  STON/O2. 3101282   7.9250   NaN        S        2  


In [ ]:
df['Sextype']=df['Sex'].replace(['male','female'],['1','2'])

方法三：
使用sklearn.preprocessing的LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
for feat in ['Cabin','Ticket']:                                         #选择需要编码的列
    lbl=LabelEncoder()
    label_dict=dict(zip(df[feat].unique(),range(df[feat].nunique())))   #把需要编码的内容按照其不重复的range（）数量进行编码
    df[feat+'_labelEncode']=df[feat].map(label_dict)                    #新增列[Cabin_labelEncode]&[Ticket_labelEncode],并且用新的编码替代其原编码
    df[feat+'_labelEncode']=lbl.fit_transform(df[feat].astype(str))     #把新编码转换为索引值
df.head()

One-hot编码：将分类数据转换为虚拟变量

In [4]:
import pandas as pd
df=pd.read_csv('D:/data/train.csv')
for feat in ['Sex','Cabin','Embarked']:
    x=pd.get_dummies(df[feat],prefix=feat)          #get_dummies()把变量变为虚拟变量，prefix是定义虚拟变量的前缀
    df=pd.concat([df,x],axis=1)                     #pd.concat()  concatenate pandas objects along a particular axis
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,0,0,0,0,0,0,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,0,0,0,0,0,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,0,0,0,0,0,0,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,0,0,0,0,0,0,0,0,1


从纯文本Name特征里提取出Titles的特征(所谓的Titles就是Mr,Miss,Mrs等)

In [10]:
import pandas as pd
df=pd.read_csv('D:/data/train.csv')
df['Title']=df['Name'].str.extract('([A-Za-z]+)\.', expand=False)     #正则表达式：[A-Za-z]+ 代表包含所有大小写字母； 因为Mr/Mrs之后总有一点.,所以通过这样的方式将它们提取出来
print (df.head())                                                     # expand=True:a pattern with one group will return a dataframe with one column; expand=False: a pattern with one group will return a Series

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked Title  
0      0         A/5 21171   7.2500   NaN        S    Mr  
1      0          PC 17599  71.2833   C85        C   Mrs  
2      0  STON/O2. 3101282   7.9250   NaN        S  Miss  
3      0            113803  53.1000  C123        S   Mrs  
4      0            37345